# Padrões de Junção Avançados

Nesta seção, abordaremos padrões de junção mais avançados que são úteis na análise de dados. Começaremos com uma breve revisão de `INNER JOIN`, `LEFT JOIN`, `RIGHT JOIN` e `FULL OUTER JOIN`.

Vamos conectar ao banco de dados `company_operations.db`.

In [ ]:
import sqlite3
import pandas as pd 

conn = sqlite3.connect('company_operations.db')


Vamos também ajustar o Pandas para exibir mais linhas e não truncá-las.

In [ ]:
pd.options.display.max_rows = 999

## INNER JOIN, LEFT, RIGHT e FULL OUTER JOIN

Usamos `INNER JOIN` e `LEFT JOIN` algumas vezes neste curso. Mas vamos revisar a diferença fundamental entre eles e, em seguida, como eles se estendem a `RIGHT JOIN` e `FULL OUTER JOIN`.

Abaixo, unimos as tabelas `CUSTOMER` e `CUSTOMER_ORDER` usando um `INNER JOIN`. Isso nos permite trazer informações de `CUSTOMER` para mostrar junto com cada `CUSTOMER_ORDER`, como `CUSTOMER_NAME` e `SHIP_ADDRESS` (que é concatenado a partir de quatro campos).

In [ ]:
sql = """
SELECT CUSTOMER_ORDER_ID,
CUSTOMER_NAME,
ADDRESS || ' ' || CITY || ', ' || STATE || ' ' || ZIP AS SHIP_ADDRESS,
ORDER_DATE,
PRODUCT_ID,
QUANTITY

FROM CUSTOMER INNER JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID
"""

pd.read_sql(sql, conn)

> Observe que `INNER JOIN` também pode ser aliasado como `JOIN`. Eu, pessoalmente, não gosto de usar esse alias. Prefiro ser explícito sobre minha intenção de usar um `INNER JOIN`. Existem muitos tipos de junções e, quando vejo uma consulta SQL usando `JOIN` em vez de `INNER JOIN`, presumo que o programador não saiba que existem vários tipos de junções.

No entanto, lembre-se de que existe um registro `CUSTOMER` sem nenhum registro `CUSTOMER_ORDER`.

In [ ]:
sql = """
SELECT * FROM CUSTOMER 
WHERE CUSTOMER_ID NOT IN (SELECT DISTINCT CUSTOMER_ID FROM CUSTOMER_ORDER)
"""

pd.read_sql(sql, conn)

Se quiséssemos um registro de espaço reservado para esse registro `CUSTOMER`, mesmo que não haja um registro `CUSTOMER_ORDER` para unir, podemos fazer isso usando um `LEFT JOIN`. Isso incluirá todos os registros na tabela "esquerda" (à esquerda das palavras-chave `LEFT JOIN`), mesmo que não haja registros para unir na tabela "direita" (à direita das palavras-chave `LEFT JOIN`). Quando não houver registros para unir na tabela "direita", esses campos da tabela "direita" serão nulos.

Vamos classificar um desses campos `CUSTOMER_ORDER` para que os registros `NULL` resultantes do `LEFT JOIN` fiquem no topo.

In [ ]:
sql = """
SELECT CUSTOMER_ORDER_ID,
CUSTOMER_NAME,
ADDRESS || ' ' || CITY || ', ' || STATE || ' ' || ZIP AS SHIP_ADDRESS,
ORDER_DATE,
PRODUCT_ID,
QUANTITY

FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

ORDER BY ORDER_DATE
"""

pd.read_sql(sql, conn)

`LEFT JOIN` é uma abreviação de `LEFT OUTER JOIN`, então você também pode usar essas palavras-chave. Observe que elas são as mesmas.

`RIGHT JOIN` (ou `RIGHT OUTER JOIN`) é exatamente o mesmo que `LEFT JOIN`, exceto que inverte a direção. Isso incluirá todos os registros na tabela `direita` (à direita das palavras-chave `RIGHT JOIN`), mesmo que não haja registros para unir na tabela `esquerda` (à direita das palavras-chave `RIGHT JOIN`). Quando não houver registros para unir na tabela `esquerda`, esses campos da tabela `esquerda` serão nulos.

Podemos obter exatamente o mesmo resultado da nossa consulta `LEFT JOIN` anterior especificando `CUSTOMER_ORDER RIGHT JOIN CUSTOMER` em vez de `CUSTOMER LEFT JOIN CUSTOMER_ORDER`.

In [ ]:
sql = """
SELECT CUSTOMER_ORDER_ID,
CUSTOMER_NAME,
ADDRESS || ' ' || CITY || ', ' || STATE || ' ' || ZIP AS SHIP_ADDRESS,
ORDER_DATE,
PRODUCT_ID,
QUANTITY

FROM CUSTOMER_ORDER RIGHT JOIN CUSTOMER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

ORDER BY ORDER_DATE
"""

pd.read_sql(sql, conn)

Como qualquer consulta `RIGHT JOIN` pode ser composta como uma `LEFT JOIN`, ela raramente é usada. É uma prática recomendada usar uma `LEFT JOIN`. A `FULL OUTER JOIN` realiza uma `LEFT JOIN` e uma `RIGHT JOIN` simultaneamente, incluindo todos os registros das tabelas "esquerda" e "direita". Como registros órfãos são frequentemente ilegais em bancos de dados relacionais (por exemplo, um filho sem um pai, uma `CUSTOMER_ORDER` sem um `CUSTOMER` existente), ela raramente é usada. No entanto, é perfeitamente aceitável ter um pai sem um filho, como visto com um registro `CUSTOMER` sem registros `CUSTOMER_ORDER`.

Também podemos unir mais de duas tabelas. Abaixo, estou unindo a uma terceira tabela chamada `PRODUCT` para obter informações do produto, como `PRODUCT_NAME` e `PRICE`. Preciso usar outro `LEFT JOIN` aqui em vez de um `INNER JOIN` porque o `PRODUCT_ID` será nulo para "Alpha Medical", já que não há pedidos. `LEFT JOIN` tolerará valores nulos em sua condição de junção, mas `RIGHT JOIN` os omitirá.

In [ ]:
sql = """
SELECT CUSTOMER_ORDER_ID,
CUSTOMER_NAME,
ADDRESS || ' ' || CITY || ', ' || STATE || ' ' || ZIP AS SHIP_ADDRESS,
ORDER_DATE,
PRODUCT.PRODUCT_ID,
PRODUCT_NAME,
PRICE,
QUANTITY

FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

LEFT JOIN PRODUCT
ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID

ORDER BY ORDER_DATE
"""

pd.read_sql(sql, conn)

Com as três tabelas unidas, podemos introduzir um `GROUP BY` e algumas expressões para encontrar a receita total por `CUSTOMER`.

In [ ]:
sql = """
SELECT 
CUSTOMER.CUSTOMER_ID,
CUSTOMER_NAME,
COALESCE(SUM(PRICE * QUANTITY), 0) AS TOTAL_REVENUE

FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

LEFT JOIN PRODUCT
ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID

GROUP BY 1, 2
"""

pd.read_sql(sql, conn)

## Tabelas Voláteis/Temporárias

Especialmente ao realizar análises de dados, pode haver momentos em que seja necessário injetar uma tabela temporária no banco de dados com alguns dados, geralmente para realizar a junção. Digamos que você tenha uma planilha de descontos que deseja conceder a determinados clientes para determinados produtos. Em vez de criar uma quantidade monstruosa de expressões `CASE` para lidar com isso, você pode criar uma **tabela temporária** que armazenará os dados no banco de dados até que você desconecte.

Ela segue exatamente a mesma convenção `CREATE TABLE`, mas você a especifica como `CREATE TEMP TABLE`. Outras plataformas, como o Teradata, podem chamá-la de `VOLATILE TABLE`.

Vamos primeiro criar um `DataFrame` do Pandas com nossos descontos.

In [ ]:
discounts = pd.DataFrame({
    'customer_id' : [2,2,2,4,4,4,7,7,7,7,7],
    'product_id' : [4,5,9,3,8,6,5,11,12,13,15],
    'discounts' : [.1, .12, .2, .1, .3, .15, .05, .12, .15, .35, .05]
})

discounts

Em seguida, vamos criar nossa tabela temporária `DISCOUNT` e carregar o `DataFrame` do Pandas nela usando `executemany()` e um modelo `INSERT`.

In [ ]:
create_sql = """
CREATE TEMP TABLE DISCOUNT ( 
    CUSTOMER_ID INTEGER NOT NULL, 
    PRODUCT_ID INTEGER NOT NULL, 
    DISCOUNT_RATE DOUBLE NOT NULL
);
"""
conn.execute(create_sql)

insert_sql = 'INSERT INTO DISCOUNT (CUSTOMER_ID, PRODUCT_ID, DISCOUNT_RATE) VALUES (?, ?, ?)'
conn.executemany(insert_sql, discounts.values)

pd.read_sql("SELECT * FROM DISCOUNT", conn)

Agora podemos entrar na tabela `DESCONTO` e aplicar os descontos!

In [ ]:
sql = """
SELECT CUSTOMER_ORDER_ID, 
CUSTOMER_ORDER.CUSTOMER_ID,
CUSTOMER_ORDER.PRODUCT_ID, 
PRICE,
DISCOUNT_RATE, 
PRICE * (1.0 - DISCOUNT_RATE) AS DISCOUNT_PRICE

FROM CUSTOMER_ORDER INNER JOIN PRODUCT
ON CUSTOMER_ORDER.PRODUCT_ID = PRODUCT.PRODUCT_ID

LEFT JOIN DISCOUNT
ON CUSTOMER_ORDER.CUSTOMER_ID = DISCOUNT.CUSTOMER_ID
AND CUSTOMER_ORDER.PRODUCT_ID = DISCOUNT.PRODUCT_ID

WHERE DISCOUNT_RATE IS NOT NULL
"""

pd.read_sql(sql, conn)

## Self Joins e Non-Equi Joins

Depois de todas as loucuras que fizemos com subconsultas e tabelas derivadas, provavelmente não é surpresa que possamos unir uma tabela a ela mesma. Isso pode ser útil para, por exemplo, obter o pedido do dia anterior `QUANTITY` em relação a cada registro. Usamos o alias `CUSTOMER_ORDER` duas vezes, como `o1` e `o2`. Pense nisso como criar duas cópias separadas dessa tabela e juntá-las.

In [ ]:
sql = """
SELECT o1.CUSTOMER_ORDER_ID,
o1.CUSTOMER_ID,
o1.PRODUCT_ID,
o1.ORDER_DATE,
o1.QUANTITY,
o2.QUANTITY AS PREV_DAY_QUANTITY

FROM CUSTOMER_ORDER o1 LEFT JOIN CUSTOMER_ORDER o2

ON o1.CUSTOMER_ID = o2.CUSTOMER_ID
AND o1.PRODUCT_ID = o2.PRODUCT_ID
AND o2.ORDER_DATE = date(o1.ORDER_DATE, '-1 day')

WHERE o1.ORDER_DATE BETWEEN '2024-03-05' AND '2024-03-11'
"""

pd.read_sql(sql, conn)

Se você quiser obter a quantidade anterior de cada registro, mesmo que seja anterior ao dia anterior, podemos fazer isso usando uma subconsulta correlacionada.

In [ ]:
sql = """
SELECT ORDER_DATE,
PRODUCT_ID,
CUSTOMER_ID,
QUANTITY,
(
    SELECT QUANTITY
    FROM CUSTOMER_ORDER c2
    WHERE c1.ORDER_DATE > c2.ORDER_DATE
    AND c1.PRODUCT_ID = c2.PRODUCT_ID
    AND c1.CUSTOMER_ID = c2.CUSTOMER_ID
    ORDER BY ORDER_DATE DESC
    LIMIT 1
) as PREV_QTY
FROM CUSTOMER_ORDER c1
"""

pd.read_sql(sql, conn)

Você também não precisa unir estritamente por igualdade. Podemos unir uma tabela a ela mesma, mas usar isso para somar registros de datas anteriores que compartilham os mesmos `PRODUCT_ID` e `CUSTOMER_ID`.

In [ ]:
sql = """
SELECT c1.ORDER_DATE,
c1.PRODUCT_ID,
c1.CUSTOMER_ID,
c1.QUANTITY,
SUM(c2.QUANTITY) as ROLLING_QTY

FROM CUSTOMER_ORDER c1 INNER JOIN CUSTOMER_ORDER c2
ON c1.PRODUCT_ID = c2.PRODUCT_ID
AND c1.CUSTOMER_ID = c2.CUSTOMER_ID
AND c1.ORDER_DATE >= c2.ORDER_DATE

GROUP BY 1, 2, 3, 4
"""

pd.read_sql(sql, conn)

Aprenderemos como aproveitar as funções de janela mais tarde, o que facilitará essas tarefas, mas ainda podemos aproveitar essas ferramentas quando precisarmos criar lógicas mais complexas com flexibilidade.

# Cross Joins 

Uma **junção cruzada** é um tipo especial de junção que cria todas as combinações possíveis entre duas ou mais tabelas. Isso também é conhecido como **junção cartesiana**. Por exemplo, podemos parear todos os `PRODUCT_ID` possíveis com todos os `CUSTOMER_ID` possíveis.

In [ ]:
sql = """
SELECT PRODUCT_ID, CUSTOMER_ID 
FROM PRODUCT CROSS JOIN CUSTOMER
"""

pd.read_sql(sql, conn)

Por que isso pode ser útil? Digamos que temos a consulta abaixo que encontra a receita total por `CUSTOMER_ID` e `PRODUCT_ID` para uma data específica.

In [ ]:
sql = """
SELECT CUSTOMER.CUSTOMER_ID, 
PRODUCT.PRODUCT_ID, 
SUM(PRICE * QUANTITY) AS TOTAL_REVENUE

FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

LEFT JOIN PRODUCT 
ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID

WHERE ORDER_DATE = '2024-03-01'
GROUP BY 1, 2 
"""

pd.read_sql(sql, conn)
## Recursive Self Joins

No entanto, queremos ver todos os clientes e produtos, mesmo que não tenha havido pedidos naquele dia para os `PRODUCT_ID` e `CUSTOMER_ID`. Podemos empacotar essa consulta em uma expressão de tabela comum e aplicar `LEFT JOIN` a essa consulta `CROSS JOIN` combinando todos os `PRODUCT_ID` e `CUSTOMER_ID`. Também aplicaremos `COALESCE()` a valores nulos para 0 quando não houver vendas.

In [ ]:
sql = """
WITH totals AS ( 
    SELECT CUSTOMER.CUSTOMER_ID, 
    PRODUCT.PRODUCT_ID, 
    SUM(PRICE * QUANTITY) AS TOTAL_REVENUE
    
    FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
    ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID
    
    LEFT JOIN PRODUCT 
    ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID
    
    WHERE ORDER_DATE = '2024-03-01'
    GROUP BY 1, 2 
), 

all_combos AS ( 
    SELECT PRODUCT_ID, CUSTOMER_ID 
    FROM PRODUCT CROSS JOIN CUSTOMER
)

SELECT all_combos.PRODUCT_ID, 
all_combos.CUSTOMER_ID,
COALESCE(totals.TOTAL_REVENUE, 0) AS TOTAL_REVENUE

FROM all_combos LEFT JOIN totals
ON all_combos.CUSTOMER_ID = totals.CUSTOMER_ID
AND all_combos.PRODUCT_ID = totals.PRODUCT_ID
"""

pd.read_sql(sql, conn)

Tenha cuidado para não criar explosões combinatórias com junções cruzadas, pois criar muitas combinações pode tornar suas consultas muito lentas.

## Self Joins Recursivos

Vamos dar uma olhada em uma tabela diferente: a tabela `EMPLOYEE`. Observe que a coluna `MANAGER_ID` possui valores que apontam para outros registros `EMPLOYEE`.

In [ ]:
pd.read_sql("SELECT * FROM EMPLOYEE", conn, index_col='ID')

Você pode autojuntar esta tabela e subir um nível, mas como subimos em toda a hierarquia? Vamos nos concentrar em um funcionário com `FIRST_NAME` "Mag" e um `ID` de `29`. Podemos usar um tipo especial de expressão de tabela comum que é `RECURSIVE`. Inicializamos com um valor inicial de `29` e adicionamos recursivamente os IDs em toda a hierarquia até que não haja mais (quando chega ao CEO).

In [ ]:
sql = """
-- generates a list of employee ID's hierarchical to Mag
WITH RECURSIVE hierarchy_of_mag(x) AS (
 SELECT 29 -- start with Mag's ID
 UNION ALL -- append each manager ID recursively
 SELECT MANAGER_ID
 FROM hierarchy_of_mag INNER JOIN EMPLOYEE
 ON EMPLOYEE.ID = hierarchy_of_mag.x -- employee ID must equal previous recursion
)

SELECT * FROM hierarchy_of_mag;
"""

pd.read_sql(sql, conn)

Podemos usar esse conjunto de IDs para qualificar esses funcionários nessa hierarquia.

In [ ]:
sql = """
-- generates a list of employee ID's hierarchical to Mag
WITH RECURSIVE hierarchy_of_mag(x) AS (
 SELECT 29 -- start with Mag's ID
 UNION ALL -- append each manager ID recursively
 SELECT MANAGER_ID 
 FROM hierarchy_of_mag INNER JOIN EMPLOYEE
 ON EMPLOYEE.ID = hierarchy_of_mag.x -- employee ID must equal previous recursion
)

SELECT * FROM EMPLOYEE WHERE ID IN hierarchy_of_mag;
"""

pd.read_sql(sql, conn)

Consultas recursivas também são úteis para gerar um intervalo de valores consecutivos, como um intervalo de números inteiros ou datas/horas. Aqui está um intervalo de números inteiros de 1 a 1000.

In [ ]:
sql = """ 
WITH RECURSIVE integers(i) AS (
    SELECT 1
        UNION ALL
    SELECT i + 1 
    FROM integers
    WHERE i < 1000
)

SELECT * FROM integers
"""

pd.read_sql(sql, conn)

E aqui está uma enumeração de datas de agora até 31 de dezembro de 2030.

In [ ]:
sql = """ 
WITH RECURSIVE calendar_dates(dt) AS (
    SELECT date('now')
        UNION ALL
    SELECT date(dt, '+1 day')
    FROM calendar_dates
    WHERE dt < '2030-12-31'
)
SELECT * FROM calendar_dates
"""

pd.read_sql(sql, conn)

Voltando ao nosso exemplo anterior de `CROSS JOIN`, podemos aproveitar essa enumeração de datas para preencher lacunas não apenas para `CUSTOMER_ID` e `PRODUCT_ID`, mas também para `ORDER_DATE`. Em outras palavras, podemos ver todos os `CUSTOMER_ID` e `PRODUCT_ID` representados em nossa consulta para cada `ORDER_DATE`, mesmo que não haja pedidos. Certifique-se de listar as consultas `RECURSIVE` primeiro em suas expressões de tabela comuns.

In [ ]:
sql = """
WITH RECURSIVE calendar_dates(dt) AS (
    SELECT date('2020-01-01')
        UNION ALL
    SELECT date(dt, '+1 day')
    FROM calendar_dates
    WHERE dt <'2099-12-31'
), 

totals AS ( 
    SELECT CUSTOMER.CUSTOMER_ID, 
    PRODUCT.PRODUCT_ID, 
    ORDER_DATE,
    SUM(PRICE * QUANTITY) AS TOTAL_REVENUE
    
    FROM CUSTOMER INNER JOIN CUSTOMER_ORDER
    ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID
    
    INNER JOIN PRODUCT 
    ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID
    
    GROUP BY 1, 2, 3
), 

all_combos AS ( 
    SELECT PRODUCT_ID, CUSTOMER_ID, dt
    FROM PRODUCT CROSS JOIN CUSTOMER 
    CROSS JOIN calendar_dates
    WHERE dt BETWEEN '2024-03-01' AND '2024-03-31'
)
SELECT all_combos.dt as ORDER_DATE, 
all_combos.PRODUCT_ID, 
all_combos.CUSTOMER_ID,
COALESCE(totals.TOTAL_REVENUE, 0) AS TOTAL_REVENUE

FROM all_combos LEFT JOIN totals
ON all_combos.CUSTOMER_ID = totals.CUSTOMER_ID
AND all_combos.PRODUCT_ID = totals.PRODUCT_ID
AND all_combos.dt = totals.ORDER_DATE
"""

pd.read_sql(sql, conn)

Vejamos um último exemplo. Observe a tabela `EMPLOYEE_AIR_TRAVEL`.

In [ ]:
pd.read_sql("SELECT * FROM EMPLOYEE_AIR_TRAVEL", conn)

Observe que a coluna `NUM_OF_PASSENGERS` indica o número de passageiros naquele bilhete. Digamos que quiséssemos dividi-los em vários registros, então um `NUM_OF_PASSENGERS` de "3" transformaria esse registro em três cópias. Podemos usar uma enumeração `RECURSIVE` de inteiros para fazer isso, copiando o registro quantas vezes precisarmos.

In [ ]:
sql = """
WITH RECURSIVE integers(i) AS (
    SELECT 1
        UNION ALL
    SELECT i + 1 
    FROM integers
    WHERE i < 100
)

SELECT BOOKING_ID, 
BOOKED_EMPLOYEE_ID,
DEPARTURE_DATE,
ORIGIN,
DESTINATION,
FARE_PRICE,
integers.i AS PASSENGER_NUMBER
FROM EMPLOYEE_AIR_TRAVEL CROSS JOIN integers
ON integers.i <= NUM_OF_PASSENGERS
"""

pd.read_sql(sql, conn)

Também podemos usar esses inteiros para enumerar cópias de registros para outra finalidade: dividir `ORIGIN` e `DESTINATION` em dois registros separados. Isso pode nos ajudar a descobrir quanto estamos gastando para que os funcionários voem por cada `AEROPORTO`, independentemente de esse `AEROPORTO` ser o `ORIGIN` ou o `DESTINATION`.

In [ ]:
sql = """
WITH RECURSIVE integers(i) AS (
    SELECT 1
        UNION ALL
    SELECT i + 1 
    FROM integers
    WHERE i < 100
)

SELECT
CASE WHEN integers.i == 1 THEN ORIGIN ELSE DESTINATION END AS AIRPORT,
SUM(FARE_PRICE * NUM_OF_PASSENGERS) AS AIRPORT_COST
FROM EMPLOYEE_AIR_TRAVEL CROSS JOIN integers
ON integers.i <= 2
GROUP BY AIRPORT
"""

pd.read_sql(sql, conn)

Como você pode ver, consultas recursivas são muito poderosas e altamente subutilizadas e subestimadas. Use-as para preencher lacunas em seus dados, duplicar e modificar registros ou simplesmente gerar um intervalo de valores, incluindo inteiros e datas/horas.

## Exercício

Para cada data do calendário e `PRODUCT_ID`, mostre a quantidade total pedida para o período de `2024-01-01` a `2024-02-28`. Já existe muita codificação padrão. Basta preencher os pontos de interrogação "?".

In [ ]:
sql = """
WITH RECURSIVE calendar_dates(dt) AS (
    SELECT date('2020-01-01')
        UNION ALL
    SELECT date(dt, '+1 day')
    FROM calendar_dates
    WHERE dt <'2099-12-31'
), 

product_totals_by_date AS ( 
    ?
), 

all_combos AS ( 
    SELECT ?, ?
    FROM PRODUCT ? ? calendar_dates
    WHERE dt BETWEEN '2024-01-01' AND '2024-02-28'
)
SELECT all_combos.dt as ORDER_DATE, 
all_combos.PRODUCT_ID, 
COALESCE(?, 0) AS TOTAL_REVENUE

FROM all_combos LEFT JOIN product_totals_by_date
ON all_combos.PRODUCT_ID = ?
AND all_combos.dt = ?
"""

pd.read_sql(sql, conn)

### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
sql = """
WITH RECURSIVE calendar_dates(dt) AS (
    SELECT date('2020-01-01')
        UNION ALL
    SELECT date(dt, '+1 day')
    FROM calendar_dates
    WHERE dt <'2099-12-31'
), 

product_totals_by_date AS ( 
    SELECT PRODUCT.PRODUCT_ID, 
    ORDER_DATE,
    SUM(PRICE * QUANTITY) AS TOTAL_REVENUE
    
    FROM PRODUCT INNER JOIN CUSTOMER_ORDER 
    ON PRODUCT.PRODUCT_ID = CUSTOMER_ORDER.PRODUCT_ID
    
    GROUP BY 1,2
), 

all_combos AS ( 
    SELECT PRODUCT_ID, dt
    FROM PRODUCT CROSS JOIN calendar_dates
    WHERE dt BETWEEN '2024-01-01' AND '2024-02-28'
)
SELECT all_combos.dt as ORDER_DATE, 
all_combos.PRODUCT_ID, 
COALESCE(product_totals_by_date.TOTAL_REVENUE, 0) AS TOTAL_REVENUE

FROM all_combos LEFT JOIN product_totals_by_date
ON all_combos.PRODUCT_ID = product_totals_by_date.PRODUCT_ID
AND all_combos.dt = product_totals_by_date.ORDER_DATE
"""

pd.read_sql(sql, conn)